In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [2]:
df = pd.read_csv('data.csv')
traits = df.columns.to_list()[2:]

In [3]:
plt.style.use('seaborn-v0_8-white')
font_size = plt.rcParams.get('font.size')
plt.rcParams.update({'font.size': int(font_size * 1.3)})


In [4]:
# Collecting p-values

df_p_values = pd.DataFrame()
k = 0

for trait in traits:
    df_small = df[['Ppd_D1', 'NLP3_3B', trait]].dropna()
    df_small['Ppd-D1a_NLP-3Bg'] = \
        df_small[trait].loc[(df_small['Ppd_D1'] == 'aa') & (df_small['NLP3_3B'] == 'GG')]
    df_small['Ppd-D1a_NLP-3Ba'] = \
        df_small[trait].loc[(df_small['Ppd_D1'] == 'aa') & (df_small['NLP3_3B'] == 'AA')]
    df_small['Ppd-D1b_NLP-3Bg'] = \
        df_small[trait].loc[(df_small['Ppd_D1'] == 'bb') & (df_small['NLP3_3B'] == 'GG')]
    df_small['Ppd-D1b_NLP-3Ba'] = \
        df_small[trait].loc[(df_small['Ppd_D1'] == 'bb') & (df_small['NLP3_3B'] == 'AA')]

    p_values = np.zeros([4, 4])

    data_tuple = (df_small['Ppd-D1a_NLP-3Bg'].dropna(), df_small['Ppd-D1a_NLP-3Ba'].dropna(),
                  df_small['Ppd-D1b_NLP-3Bg'].dropna(), df_small['Ppd-D1b_NLP-3Ba'].dropna())

    for i, data1 in enumerate(data_tuple):
        for j, data2 in enumerate(data_tuple):
            _, p_values[i, j] = mannwhitneyu(data1, data2)
    
    if k == 0: 
        print(p_values)
        k += 1
    p_values_list = []
    for i in range(4):
        for j in range(4):
            if i > j:
                p_values_list.append(p_values[i, j])
    df_p_values[trait] = p_values_list

df_p_values

[[1.00000000e+00 4.46080488e-02 1.68922013e-09 3.90442820e-02]
 [4.46080488e-02 1.00000000e+00 2.23368500e-15 5.57874836e-05]
 [1.68922013e-09 2.23368500e-15 1.00000000e+00 8.10287903e-05]
 [3.90442820e-02 5.57874836e-05 8.10287903e-05 1.00000000e+00]]


,Plant_height_[cm],Spike_length_[cm],Spikelets_per_spike,Tiller_number,Internode_1_length_[cm],Internode_2_length_[cm],Internode_3_length_[cm],Internode_4_length_[cm],Internode_5_length_[cm],Internode_6_length_[cm],...,Number_of_grains_per_spike,Grain_weight_per_spike_[g],Grain_weight_per_plant_[ln(g)],Thousand_kernel_weight_[g],Time_before_heading_[days],Time_before_anthesis_[days],Spike_density,Grains_in_spikelet,Internode_number,Harvest_index
0,4.460805e-02,2.601597e-14,2.768444e-04,0.822509,5.390992e-01,0.162585,3.327074e-04,4.930217e-01,3.641466e-01,0.228818,...,4.550310e-01,0.796820,0.637734,0.258296,4.988436e-18,3.380958e-26,9.876714e-09,5.998498e-02,0.813562,4.670550e-01
1,1.689220e-09,9.445484e-02,2.301665e-08,0.015489,3.870092e-07,0.004584,7.832777e-04,7.924895e-02,2.688785e-03,0.323706,...,2.840531e-02,0.272980,0.357828,0.000621,8.177534e-86,4.564704e-33,9.345707e-05,4.607762e-02,0.804565,8.860109e-04
2,2.233685e-15,7.550126e-23,1.451534e-20,0.027699,2.500679e-10,0.000017,6.610033e-12,1.508134e-02,4.003203e-02,0.747447,...,1.372720e-01,0.339916,0.645573,0.008593,2.174706e-159,1.377117e-91,6.869459e-02,1.428868e-04,0.625729,1.055854e-05
3,3.904428e-02,1.095352e-01,3.467238e-03,0.383910,1.536909e-01,0.300209,1.608041e-06,1.615448e-08,9.968095e-11,0.529681,...,1.088102e-02,0.000015,0.001967,0.004352,1.797762e-51,4.493204e-15,1.375461e-08,1.056472e-07,0.004406,9.214159e-08
4,5.578748e-05,3.116278e-10,4.545003e-11,0.520183,3.666398e-01,0.770536,3.499965e-17,1.456604e-10,3.556156e-08,0.102428,...,5.807671e-04,0.000012,0.006903,0.064547,2.917805e-130,1.624811e-68,9.776016e-01,2.470383e-12,0.001845,5.303240e-11
5,8.102879e-05,5.028510e-04,6.639141e-03,0.105473,3.666572e-11,0.000072,1.951835e-01,2.172844e-04,4.108358e-04,0.141611,...,8.679371e-07,0.000736,0.028959,0.435966,4.093103e-23,2.590124e-12,6.685329e-02,9.098001e-04,0.008789,8.463932e-02


In [ ]:
# Makeing Benjamini-Hochberg Procedure for Multivariate Hypothesis Testing

aray_of_p_values = np.array(df_p_values)
_, bonf_corr_pvals, _, _ = multipletests(aray_of_p_values.flatten(), method='fdr_bh')
bonf_corr_pvals = bonf_corr_pvals.reshape(aray_of_p_values.shape)
df_p_values = pd.DataFrame(bonf_corr_pvals, columns=df_p_values.columns, index=df_p_values.index)
df_p_values


,Plant_height_[cm],Spike_length_[cm],Spikelets_per_spike,Tiller_number,Internode_1_length_[cm],Internode_2_length_[cm],Internode_3_length_[cm],Internode_4_length_[cm],Internode_5_length_[cm],Internode_6_length_[cm],...,Number_of_grains_per_spike,Grain_weight_per_spike_[g],Grain_weight_per_plant_[ln(g)],Thousand_kernel_weight_[g],Time_before_heading_[days],Time_before_anthesis_[days],Spike_density,Grains_in_spikelet,Internode_number,Harvest_index
0,7.009836e-02,2.146317e-13,7.027588e-04,0.835163,5.881082e-01,0.212487,8.286296e-04,5.515158e-01,4.360041e-01,0.293243,...,0.522296,0.828191,0.684398,0.327837,5.487280e-17,5.578581e-25,4.656165e-08,9.101170e-02,0.832482,5.269338e-01
1,8.258410e-09,1.340649e-01,9.494368e-08,0.026552,1.419034e-06,0.008643,1.723211e-03,1.149545e-01,5.377569e-03,0.399339,...,0.046869,0.343175,0.433333,0.001413,2.698586e-84,8.607728e-32,2.681811e-04,7.155583e-02,0.829708,1.917269e-03
2,2.106046e-14,9.966166e-22,1.741841e-19,0.046282,1.320359e-09,0.000053,4.592234e-11,2.619390e-02,6.366540e-02,0.789304,...,0.184897,0.415453,0.687223,0.015539,2.870612e-157,6.059314e-90,1.007521e-01,3.929387e-04,0.677018,3.573659e-05
3,6.285177e-02,1.506109e-01,6.830976e-03,0.448462,2.028720e-01,0.373845,5.585828e-06,6.878682e-08,5.720820e-10,0.582649,...,0.019151,0.000047,0.004057,0.008428,3.955076e-50,3.954020e-14,6.052027e-08,3.984408e-07,0.008428,3.577262e-07
4,1.712546e-04,1.582110e-09,2.856859e-10,0.577010,4.360041e-01,0.807228,3.553811e-16,8.011324e-10,1.422463e-07,0.143835,...,0.001345,0.000040,0.012655,0.096821,1.925751e-128,4.289501e-67,9.776016e-01,1.899424e-11,0.003866,3.181944e-10
5,2.376845e-04,1.185292e-03,1.234319e-02,0.146552,2.419937e-10,0.000217,2.525905e-01,5.736307e-04,9.860060e-04,0.188815,...,0.000003,0.001647,0.047192,0.504802,6.003218e-22,1.899424e-11,9.915319e-02,1.936994e-03,0.015678,1.214390e-01


In [6]:
# Make square array of p-values

trait = 'Plant_height_[cm]'
p_values = np.zeros([4, 4])
k = 0

for i in range(4):
    for j in range(4):
        if i > j:
            p_values[i, j] = df_p_values[trait][k]
            k += 1
        if i == j:
            p_values[i, j] = 1

for i in range(4):
    for j in range(4):
        p_values[i, j] = p_values[j, i]            

p_values

array([[1.00000000e+00, 7.00983624e-02, 8.25840953e-09, 6.28517711e-02],
       [7.00983624e-02, 1.00000000e+00, 2.10604585e-14, 1.71254601e-04],
       [8.25840953e-09, 2.10604585e-14, 1.00000000e+00, 2.37684452e-04],
       [6.28517711e-02, 1.71254601e-04, 2.37684452e-04, 1.00000000e+00]])

In [7]:
for trait in traits:
    df_small = df[['Ppd_D1', 'NLP3_3B', trait]].dropna()
    df_small['Ppd-D1a_NLP-3Bg'] = \
        df_small[trait].loc[(df_small['Ppd_D1'] == 'aa') & (df_small['NLP3_3B'] == 'GG')]
    df_small['Ppd-D1a_NLP-3Ba'] = \
        df_small[trait].loc[(df_small['Ppd_D1'] == 'aa') & (df_small['NLP3_3B'] == 'AA')]
    df_small['Ppd-D1b_NLP-3Bg'] = \
        df_small[trait].loc[(df_small['Ppd_D1'] == 'bb') & (df_small['NLP3_3B'] == 'GG')]
    df_small['Ppd-D1b_NLP-3Ba'] = \
        df_small[trait].loc[(df_small['Ppd_D1'] == 'bb') & (df_small['NLP3_3B'] == 'AA')]

    violins = plt.violinplot([df_small['Ppd-D1a_NLP-3Bg'].dropna(), df_small['Ppd-D1a_NLP-3Ba'].dropna(),
                              df_small['Ppd-D1b_NLP-3Bg'].dropna(), df_small['Ppd-D1b_NLP-3Ba'].dropna()],
                             positions=[1, 2, 3, 4], showmeans=False, showextrema=False, showmedians=False)

    violins['bodies'][0].set_facecolor('#999999')
    violins['bodies'][1].set_facecolor('#99cc33')
    violins['bodies'][2].set_facecolor('#999999')
    violins['bodies'][3].set_facecolor('#99cc33')
    violins['bodies'][0].set_alpha(0.7)
    violins['bodies'][1].set_alpha(1)
    violins['bodies'][2].set_alpha(0.7)
    violins['bodies'][3].set_alpha(1)
    color1 = violins['bodies'][0].get_facecolor().flatten()
    color2 = violins['bodies'][1].get_facecolor().flatten()
    handles = [mpatches.Patch(color=color1, alpha=1), mpatches.Patch(color=color2, alpha=1)]
    labels = ['$\it{NLP3-B1[G]}$', '$\it{NLP3-B1[A]}$']

    x = np.random.rand(len(df_small['Ppd-D1a_NLP-3Bg'].dropna())) * 0.5 - 0.5 / 2
    plt.scatter(x + 1, df_small['Ppd-D1a_NLP-3Bg'].dropna(), color='black', alpha=0.25, s=1)
    x = np.random.rand(len(df_small['Ppd-D1a_NLP-3Ba'].dropna())) * 0.5 - 0.5 / 2
    plt.scatter(x + 2, df_small['Ppd-D1a_NLP-3Ba'].dropna(), color='black', alpha=0.25, s=1)
    x = np.random.rand(len(df_small['Ppd-D1b_NLP-3Bg'].dropna())) * 0.5 - 0.5 / 2
    plt.scatter(x + 3, df_small['Ppd-D1b_NLP-3Bg'].dropna(), color='black', alpha=0.25, s=1)
    x = np.random.rand(len(df_small['Ppd-D1b_NLP-3Ba'].dropna())) * 0.5 - 0.5 / 2
    plt.scatter(x + 4, df_small['Ppd-D1b_NLP-3Ba'].dropna(), color='black', alpha=0.25, s=1)

    ax = plt.subplot()
    box = ax.get_position()
    ax.set_position([box.x0 * 0.85, box.y0 * 0.75, box.width * 0.75, box.height * 1.1])
    # ax.set_ylim([45, 75])
    # ax.spines['top'].set_visible(False)
    # ax.spines['right'].set_visible(False)
    plt.legend(handles, labels, bbox_to_anchor=(1, 0.5))

    # box_props = dict(linestyle='--', linewidth=3, color='darkgoldenrod')
    flier_props = dict(marker='o', markerfacecolor='none',
                       markeredgecolor='black', markersize=5)
    median_props = dict(linestyle='-', linewidth=1, color='black')
    mean_point_props = dict(marker='.', markerfacecolor='black',
                            markeredgecolor='black', markersize=10)
    mean_line_props = dict(linestyle='--', linewidth=2.5, color='purple')

    plt.boxplot([df_small['Ppd-D1a_NLP-3Bg'].dropna(), df_small['Ppd-D1a_NLP-3Ba'].dropna(),
                 df_small['Ppd-D1b_NLP-3Bg'].dropna(), df_small['Ppd-D1b_NLP-3Ba'].dropna()],
                positions=[1, 2, 3, 4], widths=0.1, showmeans=True, notch=True, sym=None,
                showcaps=False, flierprops=flier_props, meanprops=mean_point_props, medianprops=median_props)
    
    p_values = np.zeros([4, 4])
    
    k = 0

    for i in range(4):
        for j in range(4):
            if i > j:
                p_values[i, j] = df_p_values[trait][k]
                k += 1
            if i == j:
                p_values[i, j] = 1

    for i in range(4):
        for j in range(4):
            p_values[i, j] = p_values[j, i]  

    # Define significance level
    alpha = 0.05

    # Initialize group labels
    n_groups = p_values.shape[0]
    group_labels = [''] * n_groups

    # Create homogeneous groups
    for i in range(n_groups):
        for j in range(i + 1, n_groups):
            if p_values[i, j] > alpha:
                # If groups i and j are not significantly different, assign them the same label
                if not group_labels[i]:
                    group_labels[i] = chr(65 + i)  # Assign a letter if not already assigned
                if not group_labels[j]:
                    group_labels[j] = group_labels[i]  # Assign the same letter

    # Fill in any missing labels with unique identifiers
    used_labels = set(group_labels)
    next_label = ord('A')
    for i in range(n_groups):
        if not group_labels[i]:
            while chr(next_label) in used_labels:
                next_label += 1
            group_labels[i] = chr(next_label)
            used_labels.add(chr(next_label))

    print("Group labels:", group_labels)
    print(p_values)
    
    # plt.title(trait.replace('_', ' '))
    plt.ylabel(trait.replace('_', ' '))
    plt.xticks([1.5, 3.5], ['$\it{Ppd-D1a}$', '$\it{Ppd-D1b}$'], rotation=0)
    bottom, top = ax.get_ylim()
    y_range = top - bottom
    for i, label in enumerate(group_labels):
        plt.text(i + 1, top + y_range * 0.01, label.lower(), ha='center', va='bottom', c='k')
    plt.text(0, top + y_range * 0.01, 'U-test:', ha='left', va='bottom', c='k')
    #plt.show()
    plt.savefig('Z_' + trait + '.svg')
    plt.savefig('Z_' + trait + '.png', dpi=400)
    plt.clf()


<>:27: SyntaxWarning: invalid escape sequence '\i'
<>:27: SyntaxWarning: invalid escape sequence '\i'
<>:107: SyntaxWarning: invalid escape sequence '\i'
<>:107: SyntaxWarning: invalid escape sequence '\i'
<>:27: SyntaxWarning: invalid escape sequence '\i'
<>:27: SyntaxWarning: invalid escape sequence '\i'
<>:107: SyntaxWarning: invalid escape sequence '\i'
<>:107: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Mikhail\AppData\Local\Temp\ipykernel_8252\1507223184.py:27: SyntaxWarning: invalid escape sequence '\i'
  labels = ['$\it{NLP3-B1[G]}$', '$\it{NLP3-B1[A]}$']
C:\Users\Mikhail\AppData\Local\Temp\ipykernel_8252\1507223184.py:27: SyntaxWarning: invalid escape sequence '\i'
  labels = ['$\it{NLP3-B1[G]}$', '$\it{NLP3-B1[A]}$']
C:\Users\Mikhail\AppData\Local\Temp\ipykernel_8252\1507223184.py:107: SyntaxWarning: invalid escape sequence '\i'
  plt.xticks([1.5, 3.5], ['$\it{Ppd-D1a}$', '$\it{Ppd-D1b}$'], rotation=0)
C:\Users\Mikhail\AppData\Local\Temp\ipykernel_8252\1507223184.py:

Group labels: ['A', 'A', 'B', 'A']
[[1.00000000e+00 7.00983624e-02 8.25840953e-09 6.28517711e-02]
 [7.00983624e-02 1.00000000e+00 2.10604585e-14 1.71254601e-04]
 [8.25840953e-09 2.10604585e-14 1.00000000e+00 2.37684452e-04]
 [6.28517711e-02 1.71254601e-04 2.37684452e-04 1.00000000e+00]]
Group labels: ['A', 'B', 'A', 'A']
[[1.00000000e+00 2.14631744e-13 1.34064929e-01 1.50610947e-01]
 [2.14631744e-13 1.00000000e+00 9.96616630e-22 1.58211045e-09]
 [1.34064929e-01 9.96616630e-22 1.00000000e+00 1.18529161e-03]
 [1.50610947e-01 1.58211045e-09 1.18529161e-03 1.00000000e+00]]
Group labels: ['A', 'B', 'C', 'D']
[[1.00000000e+00 7.02758766e-04 9.49436768e-08 6.83097648e-03]
 [7.02758766e-04 1.00000000e+00 1.74184053e-19 2.85685891e-10]
 [9.49436768e-08 1.74184053e-19 1.00000000e+00 1.23431915e-02]
 [6.83097648e-03 2.85685891e-10 1.23431915e-02 1.00000000e+00]]
Group labels: ['A', 'A', 'C', 'A']
[[1.         0.83516312 0.02655178 0.4484618 ]
 [0.83516312 1.         0.04628181 0.57701001]
 [0.026

<Figure size 640x480 with 0 Axes>